# Дообучение ллм 

Подгружаем 

In [76]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re


In [77]:
file = r'C:\Users\User\Desktop\GlowByte\GlowByte\train_llm_7\my_dataset.csv'
df = pd.read_csv(file, encoding='cp1251',  sep=';')

In [78]:
df

,Возраст,ФИО,Должность,Опыт работы,Мотивационное письмо,Псих портрет,Соответствие,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,28,Петрова Светлана Леонидовна,IT consultant,Образование: Бауманка. Опыт работы: 2 лет. Ком...,Я высококвалифицированный инженер машинного об...,"К сожалению, но этот профиль не подходит для з...",0.0,NaN,NaN,NaN
1,51,Сазонова Иванна Михайловна,разработчик,Образование: ННГУ. Опыт работы: 2 лет. Компани...,"Уважаемый менеджер по найму, я пишу, чтобы под...",Ниже представлены выводы по кандидату Эмили Бр...,45.0,NaN,NaN,NaN
2,36,Логинова Лукия Матвеевна,системный аналитик,Образование: МГТУ им. Баумана. Опыт работы: 14...,Я опытный профессионал с более чем 10 -летним ...,"В базе данных указано, что профессионал имеет ...",70.0,NaN,NaN,NaN
3,37,Емельянов Станислав Абрамович,system administrator,Образование: УРФУ. Опыт работы: 13 лет. Компан...,Я подаю заявку на должность старшего разработч...,**Оценка соответствия кандидата требуемым навы...,90.0,NaN,NaN,NaN
4,49,Евпраксия Филипповна Семенова,devops инженер,Образование: СибГУТИ. Опыт работы: 4 лет. Комп...,Я опытный инженер машинного обучения с докторс...,"Надо заметить, что между психологическим профи...",0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
105,35,Элеонора Эдуардовна Баранова,data engineer,"Образование: СПбГЭТУ ""ЛЭТИ"". Опыт работы: 3 ле...",Я рад подать заявку на должность старшего Java...,Поскольку информация не приведена о конкретных...,70.0,NaN,NaN,NaN
106,46,Белозерова Нинель Тимуровна,руководитель проектов,Образование: ИрГТУ. Опыт работы: 6 лет. Компан...,Я являюсь инженером по данным с сильным опытом...,Ниже представлены выводы о соответствии кандид...,70.0,NaN,NaN,NaN
107,50,Федосеев Эмиль Георгиевич,системный аналитик,Образование: СибГУТИ. Опыт работы: 5 лет. Комп...,"Я пишу, чтобы выразить свою заинтересованность...",Согласно предоставленному психологическому про...,60.0,NaN,NaN,NaN
108,57,Ширяева Ксения Наумовна,project lead,Образование: МИСиС. Опыт работы: 4 лет. Компан...,Я взволнован возможностью подать заявку на поз...,\n\nСогласно предоставленному психологическому...,80.0,NaN,NaN,NaN


In [79]:
df = df.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], errors='ignore')
df.head()

,Возраст,ФИО,Должность,Опыт работы,Мотивационное письмо,Псих портрет,Соответствие
0,28,Петрова Светлана Леонидовна,IT consultant,Образование: Бауманка. Опыт работы: 2 лет. Ком...,Я высококвалифицированный инженер машинного об...,"К сожалению, но этот профиль не подходит для з...",0.0
1,51,Сазонова Иванна Михайловна,разработчик,Образование: ННГУ. Опыт работы: 2 лет. Компани...,"Уважаемый менеджер по найму, я пишу, чтобы под...",Ниже представлены выводы по кандидату Эмили Бр...,45.0
2,36,Логинова Лукия Матвеевна,системный аналитик,Образование: МГТУ им. Баумана. Опыт работы: 14...,Я опытный профессионал с более чем 10 -летним ...,"В базе данных указано, что профессионал имеет ...",70.0
3,37,Емельянов Станислав Абрамович,system administrator,Образование: УРФУ. Опыт работы: 13 лет. Компан...,Я подаю заявку на должность старшего разработч...,**Оценка соответствия кандидата требуемым навы...,90.0
4,49,Евпраксия Филипповна Семенова,devops инженер,Образование: СибГУТИ. Опыт работы: 4 лет. Комп...,Я опытный инженер машинного обучения с докторс...,"Надо заметить, что между психологическим профи...",0.0


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Возраст               110 non-null    int64  
 1   ФИО                   110 non-null    object 
 2   Должность             110 non-null    object 
 3   Опыт работы           110 non-null    object 
 4   Мотивационное письмо  110 non-null    object 
 5   Псих портрет          110 non-null    object 
 6   Соответствие          101 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 6.1+ KB


In [81]:
df.describe()

,Возраст,Соответствие
count,110.000000,101.000000
mean,40.536364,68.029703
std,11.721287,17.941268
min,22.000000,0.000000
25%,29.250000,60.000000
50%,40.500000,70.000000
75%,51.000000,80.000000
max,60.000000,90.000000


## Выделение отдельного столбика соответствия компании в процентах и удаление этой информации из психологического портрета

в итоге сделала это ручками

In [82]:
df = df.dropna()

In [83]:
df

,Возраст,ФИО,Должность,Опыт работы,Мотивационное письмо,Псих портрет,Соответствие
0,28,Петрова Светлана Леонидовна,IT consultant,Образование: Бауманка. Опыт работы: 2 лет. Ком...,Я высококвалифицированный инженер машинного об...,"К сожалению, но этот профиль не подходит для з...",0.0
1,51,Сазонова Иванна Михайловна,разработчик,Образование: ННГУ. Опыт работы: 2 лет. Компани...,"Уважаемый менеджер по найму, я пишу, чтобы под...",Ниже представлены выводы по кандидату Эмили Бр...,45.0
2,36,Логинова Лукия Матвеевна,системный аналитик,Образование: МГТУ им. Баумана. Опыт работы: 14...,Я опытный профессионал с более чем 10 -летним ...,"В базе данных указано, что профессионал имеет ...",70.0
3,37,Емельянов Станислав Абрамович,system administrator,Образование: УРФУ. Опыт работы: 13 лет. Компан...,Я подаю заявку на должность старшего разработч...,**Оценка соответствия кандидата требуемым навы...,90.0
4,49,Евпраксия Филипповна Семенова,devops инженер,Образование: СибГУТИ. Опыт работы: 4 лет. Комп...,Я опытный инженер машинного обучения с докторс...,"Надо заметить, что между психологическим профи...",0.0
...,...,...,...,...,...,...,...
105,35,Элеонора Эдуардовна Баранова,data engineer,"Образование: СПбГЭТУ ""ЛЭТИ"". Опыт работы: 3 ле...",Я рад подать заявку на должность старшего Java...,Поскольку информация не приведена о конкретных...,70.0
106,46,Белозерова Нинель Тимуровна,руководитель проектов,Образование: ИрГТУ. Опыт работы: 6 лет. Компан...,Я являюсь инженером по данным с сильным опытом...,Ниже представлены выводы о соответствии кандид...,70.0
107,50,Федосеев Эмиль Георгиевич,системный аналитик,Образование: СибГУТИ. Опыт работы: 5 лет. Комп...,"Я пишу, чтобы выразить свою заинтересованность...",Согласно предоставленному психологическому про...,60.0
108,57,Ширяева Ксения Наумовна,project lead,Образование: МИСиС. Опыт работы: 4 лет. Компан...,Я взволнован возможностью подать заявку на поз...,\n\nСогласно предоставленному психологическому...,80.0


In [84]:
df.to_csv('df.csv', index=False)

## Предобработка данных

Выделение опыт в отдельный столбик

In [85]:
def extract_experience(text):
    experience_pattern = r'Опыт работы:\s*(\d{1,2})\s*(?:лет|года|год)'
    match = re.search(experience_pattern, text)
    
    if match:
        experience_years = int(match.group(1))
        text = re.sub(experience_pattern, '', text).strip()
        return experience_years, text
    
    return None, text

In [86]:
df['Опыт'], df['Опыт работы'] = zip(*df['Опыт работы'].apply(extract_experience))

In [87]:
df[['Опыт работы', 'Опыт']].head(10)

,Опыт работы,Опыт
0,"Образование: Бауманка. . Компании: GlowByte, Н...",2.0
1,"Образование: ННГУ. . Компании: Озон, Лукойл. С...",2.0
2,Образование: МГТУ им. Баумана. . Компании: РУС...,14.0
3,"Образование: УРФУ. . Компании: Трансконтейнер,...",13.0
4,"Образование: СибГУТИ. . Компании: Вымпелком, Т...",4.0
5,"Образование: НИУ ВШЭ. . Компании: Газпром, РУС...",7.0
6,"Образование: ТГУ. . Компании: Медси, Тинькофф....",1.0
7,"Образование: ВШЭ. . Компании: Positive, ФосАгр...",12.0
8,"Образование: НИЯУ МИФИ. . Компании: Новатэк, Т...",15.0
9,Образование: СамГТУ. . Компании: Трансконтейне...,6.0


In [88]:
df

,Возраст,ФИО,Должность,Опыт работы,Мотивационное письмо,Псих портрет,Соответствие,Опыт
0,28,Петрова Светлана Леонидовна,IT consultant,"Образование: Бауманка. . Компании: GlowByte, Н...",Я высококвалифицированный инженер машинного об...,"К сожалению, но этот профиль не подходит для з...",0.0,2.0
1,51,Сазонова Иванна Михайловна,разработчик,"Образование: ННГУ. . Компании: Озон, Лукойл. С...","Уважаемый менеджер по найму, я пишу, чтобы под...",Ниже представлены выводы по кандидату Эмили Бр...,45.0,2.0
2,36,Логинова Лукия Матвеевна,системный аналитик,Образование: МГТУ им. Баумана. . Компании: РУС...,Я опытный профессионал с более чем 10 -летним ...,"В базе данных указано, что профессионал имеет ...",70.0,14.0
3,37,Емельянов Станислав Абрамович,system administrator,"Образование: УРФУ. . Компании: Трансконтейнер,...",Я подаю заявку на должность старшего разработч...,**Оценка соответствия кандидата требуемым навы...,90.0,13.0
4,49,Евпраксия Филипповна Семенова,devops инженер,"Образование: СибГУТИ. . Компании: Вымпелком, Т...",Я опытный инженер машинного обучения с докторс...,"Надо заметить, что между психологическим профи...",0.0,4.0
...,...,...,...,...,...,...,...,...
105,35,Элеонора Эдуардовна Баранова,data engineer,"Образование: СПбГЭТУ ""ЛЭТИ"". . Компании: Mail....",Я рад подать заявку на должность старшего Java...,Поскольку информация не приведена о конкретных...,70.0,3.0
106,46,Белозерова Нинель Тимуровна,руководитель проектов,"Образование: ИрГТУ. . Компании: Яндекс, Балтик...",Я являюсь инженером по данным с сильным опытом...,Ниже представлены выводы о соответствии кандид...,70.0,6.0
107,50,Федосеев Эмиль Георгиевич,системный аналитик,"Образование: СибГУТИ. . Компании: Озон, Балтик...","Я пишу, чтобы выразить свою заинтересованность...",Согласно предоставленному психологическому про...,60.0,5.0
108,57,Ширяева Ксения Наумовна,project lead,"Образование: МИСиС. . Компании: Медси, Магнит....",Я взволнован возможностью подать заявку на поз...,\n\nСогласно предоставленному психологическому...,80.0,4.0


Стандартная обработка, удаляю пунктуацию, цифры

In [89]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    pattern_punct = r'[^\w\s%]'
    text = re.sub(pattern_punct, ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [90]:
df['ФИО'] = df['ФИО'].apply(clean_text)
df['Опыт работы'] = df['Опыт работы'].apply(clean_text)
df['Мотивационное письмо'] = df['Мотивационное письмо'].apply(clean_text)
df['Псих портрет'] = df['Псих портрет'].apply(clean_text)

In [91]:
df

,Возраст,ФИО,Должность,Опыт работы,Мотивационное письмо,Псих портрет,Соответствие,Опыт
0,28,петрова светлана леонидовна,IT consultant,образование бауманка компании glowbyte норнике...,я высококвалифицированный инженер машинного об...,к сожалению но этот профиль не подходит для за...,0.0,2.0
1,51,сазонова иванна михайловна,разработчик,образование ннгу компании озон лукойл стек tab...,уважаемый менеджер по найму я пишу чтобы подат...,ниже представлены выводы по кандидату эмили бр...,45.0,2.0
2,36,логинова лукия матвеевна,системный аналитик,образование мгту им баумана компании русал маг...,я опытный профессионал с более чем летним опыт...,в базе данных указано что профессионал имеет б...,70.0,14.0
3,37,емельянов станислав абрамович,system administrator,образование урфу компании трансконтейнер тиньк...,я подаю заявку на должность старшего разработч...,оценка соответствия кандидата требуемым навыка...,90.0,13.0
4,49,евпраксия филипповна семенова,devops инженер,образование сибгути компании вымпелком татнефт...,я опытный инженер машинного обучения с докторс...,надо заметить что между психологическим профил...,0.0,4.0
...,...,...,...,...,...,...,...,...
105,35,элеонора эдуардовна баранова,data engineer,образование спбгэту лэти компании mail ru grou...,я рад подать заявку на должность старшего java...,поскольку информация не приведена о конкретных...,70.0,3.0
106,46,белозерова нинель тимуровна,руководитель проектов,образование иргту компании яндекс балтика стек...,я являюсь инженером по данным с сильным опытом...,ниже представлены выводы о соответствии кандид...,70.0,6.0
107,50,федосеев эмиль георгиевич,системный аналитик,образование сибгути компании озон балтика стек...,я пишу чтобы выразить свою заинтересованность ...,согласно предоставленному психологическому про...,60.0,5.0
108,57,ширяева ксения наумовна,project lead,образование мисис компании медси магнит стек g...,я взволнован возможностью подать заявку на поз...,согласно предоставленному психологическому про...,80.0,4.0


Удаление стоп слов + частоповторяющихся

In [92]:
from nltk.corpus import stopwords
import nltk


nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
stopwords = set(['вывод', 'оценка', 'соответствие', 'требования', 'выводы', 'навыки', 'качества', 'кандидат', 'опыт', 
                         'работы', 'работа'])
stop_words.update(stopwords)

In [94]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]
    return ' '.join(filtered_words)

In [95]:
df['Опыт работы'] = df['Опыт работы'].apply(remove_stopwords)
df['Мотивационное письмо'] = df['Мотивационное письмо'].apply(remove_stopwords)
df['Псих портрет'] = df['Псих портрет'].apply(remove_stopwords)

In [96]:
df

,Возраст,ФИО,Должность,Опыт работы,Мотивационное письмо,Псих портрет,Соответствие,Опыт
0,28,петрова светлана леонидовна,IT consultant,образование бауманка компании glowbyte норнике...,высококвалифицированный инженер машинного обуч...,сожалению профиль подходит заданной позиции хо...,0.0,2.0
1,51,сазонова иванна михайловна,разработчик,образование ннгу компании озон лукойл стек tab...,уважаемый менеджер найму пишу подать заявку до...,ниже представлены кандидату эмили браун настой...,45.0,2.0
2,36,логинова лукия матвеевна,системный аналитик,образование мгту баумана компании русал магнит...,опытный профессионал летним опытом области нау...,базе данных указано профессионал имеет лет опы...,70.0,14.0
3,37,емельянов станислав абрамович,system administrator,образование урфу компании трансконтейнер тиньк...,подаю заявку должность старшего разработчика j...,соответствия кандидата требуемым навыкам должн...,90.0,13.0
4,49,евпраксия филипповна семенова,devops инженер,образование сибгути компании вымпелком татнефт...,опытный инженер машинного обучения докторской ...,заметить психологическим профилем опытом ряд с...,0.0,4.0
...,...,...,...,...,...,...,...,...
105,35,элеонора эдуардовна баранова,data engineer,образование спбгэту лэти компании mail ru grou...,рад подать заявку должность старшего java deve...,поскольку информация приведена конкретных треб...,70.0,3.0
106,46,белозерова нинель тимуровна,руководитель проектов,образование иргту компании яндекс балтика стек...,являюсь инженером данным сильным опытом област...,ниже представлены соответствии кандидата требо...,70.0,6.0
107,50,федосеев эмиль георгиевич,системный аналитик,образование сибгути компании озон балтика стек...,пишу выразить заинтересованность должности ста...,согласно предоставленному психологическому про...,60.0,5.0
108,57,ширяева ксения наумовна,project lead,образование мисис компании медси магнит стек g...,взволнован возможностью подать заявку позицию ...,согласно предоставленному психологическому про...,80.0,4.0


Что по балансу? 

In [102]:
class_counts = df['Соответствие'].value_counts()
print(class_counts)

Соответствие
70.0    27
80.0    25
60.0    18
85.0     8
90.0     5
40.0     4
75.0     3
0.0      2
30.0     2
50.0     2
45.0     1
10.0     1
65.0     1
20.0     1
86.0     1
Name: count, dtype: int64


Разобью по категориям и буду дальше с ними работать

In [111]:
def categorize_by_range(score):
    if score <= 60:
        return 'low'
    elif score <= 75:
        return 'medium'
    else:
        return 'high'

In [169]:
df['категория'] = df['Соответствие'].apply(categorize_by_range)
df['категория'].value_counts()

категория
high      39
low       31
medium    31
Name: count, dtype: int64

Буду резать

In [170]:
df_medium = df[df['категория'] == 'medium']  
df_high = df[df['категория'] == 'high']  
df_low = df[df['категория'] == 'low'] 

In [172]:
df_balanced = pd.concat([df_medium, df_high, df_low], ignore_index=True)

In [173]:
df_balanced

,Возраст,ФИО,Должность,Опыт работы,Мотивационное письмо,Псих портрет,Соответствие,Опыт,категория
0,36,логинова лукия матвеевна,системный аналитик,образование мгту баумана компании русал магнит...,опытный профессионал летним опытом области нау...,базе данных указано профессионал имеет лет опы...,70.0,14.0,medium
1,31,дьячков поликарп владиленович,администратор баз данных,образование мгхпа строганова компании киви рос...,пишу выразить свой интерес позиции сетевого ин...,предположим компания advanced networks ищет се...,75.0,9.0,medium
2,48,глеб викторович овчинников,devops engineer,образование ргсу компании лукойл мтс стек spar...,пишу выразить свой интерес позиции ученых данн...,соответствия кандидата позиции компании общее ...,70.0,12.0,medium
3,49,олимпиада даниловна филиппова,аналитик,образование мгоу компании вымпелком согаз стек...,являюсь опытным инженером данных доктором наук...,ниже представлен анализ потенциала кандидата к...,70.0,12.0,medium
4,33,адриан феофанович уваров,test engineer,,eболее летним опытом технологической индустрии...,демонстрирует солидный технологической индустр...,70.0,NaN,medium
...,...,...,...,...,...,...,...,...,...
96,54,никонов платон тимурович,web developer,образование мгхпа строганова компании лента ба...,пишу выразить свой интерес позиции старшего уч...,поскольку заданная позиция старший ученый данн...,60.0,12.0,low
97,48,аксенов феликс юльевич,сетевой инженер,образование ниу вшэ компании яндекс трансконте...,пишу выразить заинтересованность должности ста...,основе предоставленного психологического профи...,60.0,6.0,low
98,59,феврония сергеевна анисимова,тестировщик,образование нгту компании транснефть altсибэне...,уважаемый менеджер найму пишу выразить заинтер...,заметить профессиональный психологический проф...,40.0,6.0,low
99,25,русаков эрнест данилович,devops инженер,образование тпу компании трансконтейнер аэрофл...,рад подать заявку должность старшего разработч...,оценивая психологический профиль кандидата при...,40.0,10.0,low


Разделение на выборку

In [118]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [175]:
X = df_balanced[['Псих портрет']]
y = df_balanced['категория']

In [176]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

Токенизация

In [123]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW

In [124]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(set(y_train)))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Аугментация

In [137]:
import nlpaug.augmenter.word as naw

In [139]:
contextual_aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-multilingual-cased', action="substitute"
)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

c:\Users\User\miniconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [140]:
def augment_text(text, n=1):
    augmented_texts = []
    for _ in range(n):
        augmented_text = contextual_aug.augment(text)
        if augmented_text:  
            augmented_texts.append(augmented_text)
    return augmented_texts

In [182]:
augmented_texts = []
for text in X_train['Псих портрет']:
    augmented_texts.extend(augment_text(text, n=3))

In [183]:
print("Количество аугментированных текстов:", len(augmented_texts))

Количество аугментированных текстов: 240


In [190]:
X_train_augmented = X_train[['Псих портрет']].copy()
augmented_df = pd.DataFrame({'Псих портрет': augmented_texts})
X_train_augmented = pd.concat([X_train_augmented, augmented_df], ignore_index=True)

In [147]:
import numpy as np

In [191]:
if isinstance(y_train, np.ndarray):
    y_train = pd.Series(y_train)

y_train_augmented = pd.concat([y_train] * 4, ignore_index=True)

In [192]:
X_test['Псих портрет']

84    сильные стороны имеет значительный области раз...
55    основе предоставленного психологического профи...
66    основе психологического профиля опыта сделаю с...
67    имеет необходимые области разработки данных со...
45    имеет летний ролях поддержки клиентов подходит...
39    instagram указаны тексте предположим таковы гл...
22    сильные стороны пять лет опыта sql администрир...
44    результаты оценки соответствия кандидата требо...
10    используя предоставленный психологический проф...
0     базе данных указано профессионал имеет лет опы...
18    ниже представлена кандидата позициям сетевой и...
30    ниже представлены соответствии кандидата требо...
97    основе предоставленного психологического профи...
33    соответствия кандидата позиции имеет подходящи...
77    психологический профиль кандидата демонстрируе...
4     демонстрирует солидный технологической индустр...
93    ниже приведены краткие насколько соответствует...
78    сложная задача поскольку информация компан

In [193]:
X_train_augmented['Псих портрет']

0      оценить подходит заданной позиции компании нам...
1      имеет необходимые технические роли старшего ин...
2      основе предоставленного психологического профи...
3      сожалению профиль подходит заданной позиции хо...
4      предоставлена информация требованиях компании ...
                             ...                        
315    [компании докторская степень из компьютерных н...
316    [компании докторская степень от компьютерных т...
317    [учитывая предоставленный психологический проф...
318    [учитывая предоставленный психологический проф...
319    [учитывая предоставленный психологический проф...
Name: Псих портрет, Length: 320, dtype: object

In [194]:
X_train_augmented['Псих портрет'] = X_train_augmented['Псих портрет'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
X_test['Псих портрет'] = X_test['Псих портрет'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [195]:
X_train_augmented['Псих портрет']

0      оценить подходит заданной позиции компании нам...
1      имеет необходимые технические роли старшего ин...
2      основе предоставленного психологического профи...
3      сожалению профиль подходит заданной позиции хо...
4      предоставлена информация требованиях компании ...
                             ...                        
315    компании докторская степень из компьютерных на...
316    компании докторская степень от компьютерных те...
317    учитывая предоставленный психологический профи...
318    учитывая предоставленный психологический профи...
319    учитывая предоставленный психологический профи...
Name: Псих портрет, Length: 320, dtype: object

In [196]:
train_encodings = tokenizer(X_train_augmented['Псих портрет'].tolist(), padding=True, truncation=True, max_length=256, return_tensors="pt")
test_encodings = tokenizer(X_test['Псих портрет'].tolist(), padding=True, truncation=True, max_length=256, return_tensors="pt")

In [197]:
train_labels = torch.tensor(y_train_augmented.tolist())
test_labels = torch.tensor(y_test.tolist())

In [200]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [203]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Старый код, потом немного изменила логику, но в комментах, так как потом хочу доработать с ним

In [ ]:
# X_train_combined = X_train['Опыт работы'] + " " + X_train['Мотивационное письмо'] + " " + X_train['Псих портрет']
# X_test_combined = X_test['Опыт работы'] + " " + X_test['Мотивационное письмо'] + " " + X_test['Псих портрет']

In [ ]:
# train_encodings = tokenizer(X_train_combined.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")
# test_encodings = tokenizer(X_test_combined.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
# X_train_psych = X_train['Псих портрет'] #
# X_test_psych = X_test['Псих портрет']

In [ ]:
# train_encodings = tokenizer(X_train_psych.tolist(), padding=True, truncation=True, max_length=192, return_tensors="pt") #
# test_encodings = tokenizer(X_test_psych.tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt")

Начинает дооблучение...

In [ ]:
train_labels = torch.tensor(y_train.tolist()) #
test_labels = torch.tensor(y_test.tolist())

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels) #
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [217]:
train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=False)

In [205]:
optimizer = AdamW(
    model.parameters(),
    lr=3e-5,            
    weight_decay=0.01,  
    eps=1e-8            
)

c:\Users\User\miniconda3\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [219]:
from sklearn.metrics import classification_report

best_val_loss = float('inf')
patience = 2  # Буду останавливать, если не было улучшений
patience_counter = 0

for epoch in range(8):
    total_loss = 0
    model.train()
    
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    
    all_predictions = []
    all_labels = []

    model.eval()
    val_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

            all_predictions.extend(predictions.cpu().numpy())  
            all_labels.extend(labels.cpu().numpy()) 
    
    avg_val_loss = val_loss / len(test_loader)
    accuracy = correct_predictions / total_predictions
    
    print(f'Эпоха {epoch + 1} | Тренировочная потеря: {avg_train_loss:.4f} | Валидационная потеря: {avg_val_loss:.4f} | Точность: {accuracy:.4f}')
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1
    
    if patience_counter >= patience:
        print("Ранняя остановка на эпохе:", epoch + 1)
        break


Эпоха 1 | Тренировочная потеря: 1.1074 | Валидационная потеря: 1.0942 | Точность: 0.3810
Эпоха 2 | Тренировочная потеря: 1.1029 | Валидационная потеря: 1.0950 | Точность: 0.3810
Эпоха 3 | Тренировочная потеря: 1.0982 | Валидационная потеря: 1.0948 | Точность: 0.3810
Ранняя остановка на эпохе: 3


Буду брать другую модель, эта слишком ```distilbert-base-multilingual-cased``` слабая

Как будто бы слишком мала данных и она не обучится...

Взяла другую модель ```roberta``` -> ноут умер

потом ожил

Началось снижение трейн лос, но очень слабое, с валидацией тоже самое, то есть модель не сильно обучается

Метрики я вывела, но они настолько печальные, что я решила удалить